# 1、分类数据

In [1]:
import numpy as np
import pandas as pd

In [2]:
values = pd.Series(['apple','orange','apple','apple']*2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [3]:
values.unique()

array(['apple', 'orange'], dtype=object)

In [4]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [5]:
values.value_counts()

apple     6
orange    2
dtype: int64

In [6]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

* 许多数据系统（数据仓库、统计计算或其它应用）都发展出了特定的表征重复值的方法，以进行高效的存储和计算。在数据仓库中，最好的方法是使用所谓的包含不同值的维表(Dimension Table)，将主要的参数存储为引用维表整数键：

In [7]:
values = pd.Series([0,1,0,0]*2)
dim = pd.Series(['apple','orange'])
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [8]:
dim

0     apple
1    orange
dtype: object

In [9]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

* 上述，用整数表示的方法称为分类或字典编码表示法。不同值得数组称为分类、字典或数据级。表示分类的整数值称为分类编码或简单地称为编码。

## pandas 的分类类型

In [10]:
fruits = ['apple','orange','apple','apple']* 2
N = len(fruits)
df = pd.DataFrame({'fruit':fruits,
                  'basket_id':np.arange(N),
                  'count':np.random.randint(3,15,size=N),
                  'weight':np.random.uniform(0,4,size=N)},
                 columns=['basket_id','fruit','count','weight'])
df

,basket_id,fruit,count,weight
0,0,apple,4,0.617721
1,1,orange,14,0.404439
2,2,apple,14,0.414985
3,3,apple,9,1.652178
4,4,apple,12,1.007498
5,5,orange,9,3.743293
6,6,apple,6,2.112970
7,7,apple,7,3.536211


In [14]:
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

* fruit_cat的值不是NumPy数组，而是一个pandas.Categorical实例,
* 分类对象categories 和 codes属性

In [18]:
c = fruit_cat.values
type(c)

pandas.core.arrays.categorical.Categorical

In [19]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [20]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

* 也可以直接从其他的python序列创建Categorical：

In [21]:
my_categories = pd.Categorical(['foo','bar','baz','foo','foo'])
my_categories

[foo, bar, baz, foo, foo]
Categories (3, object): [bar, baz, foo]

* 如果已经从其它源获得了分类编码，还可以使用from_codes构造器：

In [22]:
categories = ['foo','bar','baz']
codes = [0,1,2,0,0,1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [23]:
ordered_cats = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cats

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

* 输出[foo < bar < baz]指明‘foo’位于‘bar’的前面，以此类推。无序的分类实例可以通过as_ordered排序：

In [24]:
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

# ２、用分类进行计算

In [25]:
np.random.seed(12345)
draws = np.random.randn(1000)
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [27]:
#计算这个数据的分位面元，提取一些统计信息：
bins = pd.qcut(draws,4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [28]:
#可以使用labels参数qcut
bins = pd.qcut(draws, 4, labels=['Q1','Q2','Q3','Q4'])
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [32]:
#加上标签的面元分类不包含数据面元边界的信息，因此可以使用groupby提取一些汇总信息：
bins = pd.Series(bins,name='quartile')
results = (pd.Series(draws).groupby(bins).agg(['count','min','max']).reset_index())
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


# 3、使用分类提高性能
* 来看一些包含一千万元素的Series，和一些不同的分类：

In [33]:
N = 1000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo','bar','baz','qux']*(N//4))
categories = labels.astype('category')
labels.memory_usage()

4000042

In [34]:
categories.memory_usage()

1000154

In [36]:
#转换为分类不是没有代价的，但这是一次性的代价:
%time _ = labels.astype('category')

Wall time: 96.7 ms


* GroupBy使用分类操作明显更快，是因为底层的算法使用整数编码数组，而不是字符串数组。

# 4、分类方法
* 包含分类数据的Series有一些特殊的方法，类似于Series.str字符串方法。它还提供了方便的分类和编码的使用方法。看下面的Series：

In [37]:
s = pd.Series(['a','b','c','d']*2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [38]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [39]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [42]:
'''假设我们知道这个数据的实际分类集，超出了数据中的四个值。我们可以使用set_categories方法改变它们：'''
actual_categories = ['a','b','c','d','e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [43]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

* 在大数据集中，分类经常作为节省内存和高性能的便捷工具。过滤完大DataFrame或Series之后，许多分类可能不会出现在数据中。可以使用remove_unused_categories方法删除没看到的分类：

In [44]:
cat_s3 = cat_s[cat_s.isin(['a','b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [45]:
cat_s.isin(['a','b'])

0     True
1     True
2    False
3    False
4     True
5     True
6    False
7    False
dtype: bool

In [46]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

<img src='https://upload-images.jianshu.io/upload_images/7178691-6c602152c2bba658.png'>